# SPEI 干旱指数数据提取与国家级聚合（Jupyter Notebook 示例）

本Notebook演示如何将SPEI NetCDF文件按国家边界聚合为“每国每月”数据，适合后续与迁移流等数据合并建模。

In [11]:
# 1. 导入所需库
import xarray as xr
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point

# 2. 读取 SPEI NetCDF 文件
nc_path = '../data/raw/spei03.nc'
ds = xr.open_dataset(nc_path)
print(ds)

# 3. 读取国家边界矢量文件（shapefile）
# 推荐使用 Natural Earth 1:50m Admin 0 Countries
shapefile_path = '../data/raw/ne_countries.shp'
countries = gpd.read_file(shapefile_path)
countries = countries.to_crs('EPSG:4326')  # 保证坐标系一致
countries.head()

ValueError: found the following matches with the input file in xarray's IO backends: ['netcdf4', 'h5netcdf']. But their dependencies may not be installed, see:
https://docs.xarray.dev/en/stable/user-guide/io.html 
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html